# Ignore Warnings

In [34]:
import warnings
warnings.filterwarnings("ignore")

# Import Libraries

In [35]:
import pandas as pd
import numpy as np
from sklearn import datasets
# from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression as Skl_lg_reg
from from_scratch import LogisticRegression as My_lg_reg
from with_pytorch import LogisticRegression as Torch_lg_reg
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import torch
from torch.autograd import Variable

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Dataset Initialization

In [36]:
df = datasets.load_breast_cancer()
x, y = df.data, df.target

# Split into test and train datasets

In [37]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)

# Function to compute accuracy of model

In [38]:
def get_accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

# Testing model implemented from scratch

In [39]:
my_model = My_lg_reg()
my_model.fit(x_train, y_train)
y_pred = my_model.predict(x_test)
acc_my_model = get_accuracy(y_test, y_pred)

# Testing by Skl model

In [40]:
skl_model = Skl_lg_reg()
model_fit = skl_model.fit(x_train, y_train)
y_pred = model_fit.predict(x_test)
acc_skl_model = get_accuracy(y_test, y_pred)

# Testing pytorch implemented model

### Create model with init params

In [50]:
input_size = x_train.shape[1]
output_size = 1
learning_rate = 0.001
epochs = 100

model_torch = Torch_lg_reg(input_size, output_size)

### Optimize and train the model

In [51]:
criterion = torch.nn.BCELoss()
optimizer  = torch.optim.Adam(model_torch.parameters(), lr=learning_rate)

In [52]:
for epoch in range(epochs):
    inputs = Variable(torch.from_numpy(x_train))
    labels = Variable(torch.from_numpy(y_train))

In [53]:
for epoch in range(epochs):
    outputs = model_torch(inputs.float())
    loss = criterion(outputs, labels.reshape(-1, 1).float())
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

In [58]:
with torch.no_grad():
    y_predicted_by_torch = model_torch( Variable(torch.from_numpy(x_test)).float() ).data.numpy().round()

In [71]:
acc_torch_model = get_accuracy(y_test, y_predicted_by_torch)

# Imlement using Tensorflow

In [61]:
model = Sequential()
model.add(Dense(8, input_dim=input_size, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(output_size, activation='sigmoid'))

In [62]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 248       
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 5         
Total params: 289
Trainable params: 289
Non-trainable params: 0
_________________________________________________________________


In [63]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [67]:
model.fit(x=x_test, y=y_test, epochs=512, verbose=0)

In [68]:
y_pred_by_tf = model.predict(x_test).round()

In [70]:
acc_tf_model = get_accuracy(y_test, y_pred_by_tf)

# Print Results

In [74]:
print(f'Accuracy by model from scratch: {acc_my_model}')
print(f'Accuracy by SKL model: {acc_skl_model}')
print(f'Accuracy by torch model: {acc_torch_model}')
print(f'Accuracy by SKL model: {acc_tf_model}')

Accuracy by model from scratch: 0.8947368421052632
Accuracy by SKL model: 0.9385964912280702
Accuracy by torch model: 60.36842105263158
Accuracy by SKL model: 57.63157894736842
